In [1]:
%load_ext autoreload
%autoreload 2

from utils import load_config, set_environment_variables, num_tokens_from_messages, get_git_files, get_data_files
import sys
import os
config       = load_config('config.yaml')
set_environment_variables(config)
ACCESS_TOKEN = os.environ['GIT_ACCESS_TOKEN']
sys.path.append("/database")

In [2]:
from database import get_documentation_by_url, put_new_repository_documentation, get_file_documentation, delete_documentation_by_url, get_status, get_status_by_file
from repo_processor import process_file, parallel_process_files, download_and_process_repo_url
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.document_loaders import GithubFileLoader
from langchain.callbacks import get_openai_callback
from ConfluenceChain import ConfluenceChain, Parser
from pymongo import MongoClient
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pandas as pd
import nest_asyncio
import shutil
nest_asyncio.apply()


In [3]:
repo_url              = "https://github.com/Adarsh9616/Electricity_Billing_System/"
# repo_url              = "https://github.com/thuml/Time-Series-Library/"
supported_languages   = ['python', 'java', 'javascript']

# data_dict             = download_and_process_repo_url(repo_url, supported_languages)

In [31]:
# import boto3
# sqs       = boto3.client('sqs', 'us-east-1')
# queue_url = 'https://sqs.us-east-1.amazonaws.com/018192622412/doc-gen'
# response  = sqs.send_message(
#     QueueUrl=queue_url,
#     MessageBody=str("https://github.com/Adarsh9616/Electricity_Billing_System/")
# )

# """ or """

# https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url=https://github.com/Adarsh9616/Electricity_Billing_System/

In [30]:
# response = sqs.receive_message(
#     QueueUrl=queue_url,
#     MaxNumberOfMessages=1,
#     WaitTimeSeconds=1 # Long polling
# )

# messages = response.get('Messages', [])
# print(messages)
# while len(messages) != 0:
#     response = sqs.receive_message(
#         QueueUrl=queue_url,
#         MaxNumberOfMessages=1,
#         WaitTimeSeconds=1 # Long polling
#     )

#     messages = response.get('Messages', [])
#     print(messages)

[{'MessageId': '907db49d-4915-4b94-b643-bed34220873c', 'ReceiptHandle': 'AQEBPOEwO9zDxF4fb+4Av8xejsokvQWSp2x6ceBivpdwfZI9SReJic4X/GbJEpqBwYK+FmvRn1bYEEbCQwWdwnWjJrG3vxiQwyPNk0JqenT9mjMJm1YXqWQ+NeHvQ/GvK3O9PGpxisrC/TigpvIpntgImzFbA+u/R1puRfGf9kzgdLNalyJkOJanM8Dc46QfT37KB+Qch4J+BY97QklQmo//Xs9sWbfUivJxbarHCMi0isYsRa9L0Vjh2wTOYDIcBw1/tLDOySfWejeH7knNVUPdwsHMXpwtyK3mUm77jqJ2KBvqrfOHMJ73RKsjn0tn6mTc+LjFHPac4YKj2otWQmZDtTXER+PF+dKgey/RHojEKaKEnnm/LVkIn+PmstAbd2ER', 'MD5OfBody': '5e8c8de0e3a0c052ec15a8ed582f785c', 'Body': 'https://github.com/Adarsh9616/Electricity_Billing_System/'}]
[]


In [39]:
status_url = get_status(repo_url)
# get_status_by_file(repo_url, file_name)
status_url

{'repository_url': 'https://github.com/Adarsh9616/Electricity_Billing_System/',
 'overall_status': 'In progress',
 'file_level_status': {'src/splash.java': 'Completed',
  'src/generate.bill.java': 'Completed',
  'src/pay.bill.java': 'Completed',
  'src/conn.java': 'Completed',
  'src/customer.details.java': 'Completed',
  'src/login.java': 'Completed',
  'src/new.customer.java': 'Completed'}}

In [37]:
docs = get_documentation_by_url(repo_url)
docs.files

Now the file path is src/splash.java
Now the file path is src/generate.bill.java
Now the file path is src/pay.bill.java
Now the file path is src/conn.java
Now the file path is src/customer.details.java


{'src/splash.java': FileConfluenceOutput(file_path='src/splash.java', overall_summary='The file defines a Java application that creates a splash screen for an Electricity Billing System. It uses a JFrame to display an animated splash screen before transitioning to a login screen.', packages={'java.awt': PackageDetail(usage='Used for creating and managing GUI components, such as layouts and images.', description='Provides classes for creating user interfaces and for painting graphics and images.'), 'javax.swing': PackageDetail(usage='Used for creating the GUI components, such as JFrame and JLabel, and managing threading with Swing.', description="Provides a set of 'lightweight' (all-Java language) components that, to the maximum degree possible, work the same on all platforms.")}, functions={'main': FunctionDetail(name='main', description='The entry point of the application. It creates and displays the splash screen with an animation effect.', class_declaration='public static void main(

In [38]:
# delete_documentation_by_url(repo_url)

In [ ]:
# TODO: Add code to check if the documentation is already present in the database

In [45]:
from OverviewChain import OverviewParser, ConfluenceOverviewChain

In [ ]:
total_tokens = num_tokens_from_messages([{"message":  str(docs.files)}], model="gpt-4-32k-0613") 
total_tokens

In [46]:
name_of_repo = repo_url.split("/")[-2]
res          = ConfluenceOverviewChain.invoke({"name_of_repo": name_of_repo, "repo_data": str(docs.files), "output_instructions": OverviewParser.get_format_instructions()})

2024-04-19 16:49:01,620 - INFO - HTTP Request: POST https://lumnis.openai.azure.com//openai/deployments/gpt4-preview/chat/completions?api-version=2024-03-01-preview "HTTP/1.1 200 OK"


In [49]:
res

{'overall_summary': 'The Electricity Billing System project is a Java Swing GUI application designed to manage electricity billing operations. It includes functionalities for displaying a splash screen, generating and paying electricity bills, and managing customer details.',
 'functionalities': {'User Authentication': 'Not explicitly mentioned but implied through the presence of a login screen transition from the splash screen.',
  'Customer Management': 'Allows for the display and printing of customer details fetched from the database.',
  'Bill Generation': 'Users can generate electricity bills for customers based on meter number and month.',
  'Bill Payment': 'Provides a GUI for bill payment through an embedded Paytm webpage.'},
 'project_components': {'Splash Screen Class': 'Handles the display of a splash screen when the application starts.',
  'Generate Bill Class': 'Manages the generation of electricity bills through a GUI.',
  'Pay Bill Class': 'Displays a GUI for paying elect

2109